Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Auto ML : Remote Execution with Text data from Blobstorage

In this example we use the [Burning Man 2016 dataset](https://innovate.burningman.org/datasets-page/) to showcase how you can use the AutoML Classifier to handle text data from a Azure blobstorage.

Make sure you have executed the [00.configuration](00.configuration.ipynb) before running this notebook.

In this notebook you would see
1. Creating an Experiment using an existing Workspace
2. Attaching an existing DSVM to a workspace
3. Instantiating AutoMLConfig 
4. Training the Model using the DSVM
5. Exploring the results
6. Testing the fitted model

In addition this notebook showcases the following features
- **Parallel** Executions for iterations
- Asyncronous tracking of progress
- **Cancelling** individual iterations or the entire run
- Retrieving models for any iteration or logged metric
- specify automl settings as **kwargs**
- handling **text** data with **preprocess** flag


## Create Experiment

As part of the setup you have already created a <b>Workspace</b>. For AutoML you would need to create an <b>Experiment</b>. An <b>Experiment</b> is a named object in a <b>Workspace</b>, which is used to run experiments.

In [ ]:
import logging
import os
import random

from matplotlib import pyplot as plt
from matplotlib.pyplot import imshow
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import datasets

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.run import AutoMLRun

In [ ]:
ws = Workspace.from_config()

# choose a name for the run history container in the workspace
experiment_name = 'automl-remote-dsvm-blobstore-3'
# project folder
project_folder = './sample_projects/automl-remote-dsvm-blobstore-3'

experiment = Experiment(ws, experiment_name)

output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Project Directory'] = project_folder
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
pd.DataFrame(data=output, index=['']).T

## Diagnostics

Opt-in diagnostics for better experience, quality, and security of future releases

In [ ]:
from azureml.telemetry import set_diagnostics_collection
set_diagnostics_collection(send_diagnostics=True)

## Attach a Remote Linux DSVM
To use remote docker commpute target:
1. Create a Linux DSVM in Azure. Here is some [quick instructions](https://docs.microsoft.com/en-us/azure/machine-learning/desktop-workbench/how-to-create-dsvm-hdi). Make sure you use the Ubuntu flavor, NOT CentOS.  Make sure that disk space is available under /tmp because AutoML creates files under /tmp/azureml_runs. The DSVM should have more cores than the number of parallel runs that you plan to enable. It should also have at least 4Gb per core.
2. Enter the IP address, username and password below

**Note**: By default SSH runs on port 22 and you don't need to specify it. But if for security reasons you can switch to a different port (such as 5022), you can append the port number to the address. [Read more](https://render.githubusercontent.com/documentation/sdk/ssh-issue.md) on this.

In [ ]:
from azureml.core.compute import RemoteCompute

# Add your VM information below
dsvm_name     = 'mydsvm1'
dsvm_ip_addr  = '<<ip_addr>>'
dsvm_username = '<<username>>'
dsvm_password = '<<password>>'

dsvm_compute = RemoteCompute.attach(workspace=ws, name=dsvm_name, address=dsvm_ip_addr, username=dsvm_username, password=dsvm_password, ssh_port=22)

## Create a RunConfiguration with DSVM name
Run the below code to tell the runconfiguration the name of your dsvm.

In [ ]:
from azureml.core.runconfig import RunConfiguration

run_config = RunConfiguration()
run_config.target = dsvm_compute

## Change index to use master packages
If you want to use master rather than preview run the below code. Once Public preview is launched we would not need this cell.

In [ ]:
from azureml.core.runconfig import CondaDependencies

cd = CondaDependencies()

cd.remove_pip_option(pip_option="--index-url https://azuremlsdktestpypi.azureedge.net/sdk-release/Preview/E7501C02541B433786111FE8E140CAA1")
cd.set_pip_index_url(index_url="--extra-index-url https://azuremlsdktestpypi.azureedge.net/sdk-release/master/588E708E0DF342C4A80BD954289657CF")

run_config.environment.python.conda_dependencies = cd

## Create Get Data File
For remote executions you should author a get_data.py file containing a get_data() function. This file should be in the root directory of the project. You can encapsulate code to read data either from a blob storage or local disk in this file.

The *get_data()* function returns a [dictionary](README.md#getdata).

In [ ]:
if not os.path.exists(project_folder):
    os.makedirs(project_folder)

In [ ]:
%%writefile $project_folder/get_data.py

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

def get_data():
    # Burning man 2016 data
    df = pd.read_csv("https://automldemods.blob.core.windows.net/datasets/PlayaEvents2016,_1.6MB,_3.4k-rows.cleaned.2.tsv",
                     delimiter="\t", quotechar='"')
    # get integer labels
    le = LabelEncoder()
    le.fit(df["Label"].values)
    y = le.transform(df["Label"].values)
    df = df.drop(["Label"], axis=1)

    df_train, _, y_train, _ = train_test_split(df, y, test_size=0.1, random_state=42)

    return { "X" : df, "y" : y }

### View data

You can execute the *get_data()* function locally to view the *train* data

In [ ]:
%run  $project_folder/get_data.py
data_dict = get_data()
df = data_dict["X"]
y = data_dict["y"]
pd.set_option('display.max_colwidth', 15)
df['Label'] = pd.Series(y, index=df.index)
df.head()

## Instantiate AutoML <a class="anchor" id="Instatiate-AutoML-Remote-DSVM"></a>

You can specify automl_settings as **kwargs** as well. Also note that you can use the get_data() symantic for local excutions too. 

<i>Note: For Remote DSVM and Batch AI you cannot pass Numpy arrays directly to the fit method.</i>

|Property|Description|
|-|-|
|**primary_metric**|This is the metric that you want to optimize.<br> Classification supports the following primary metrics <br><i>accuracy</i><br><i>AUC_weighted</i><br><i>balanced_accuracy</i><br><i>average_precision_score_weighted</i><br><i>precision_score_weighted</i>|
|**max_time_sec**|Time limit in seconds for each iterations|
|**iterations**|Number of iterations. In each iteration Auto ML Classifier trains the data with a specific pipeline|
|**n_cross_validations**|Number of cross validation splits|
|**concurrent_iterations**|Max number of iterations that would be executed in parallel.  This should be less than the number of cores on the DSVM
|**preprocess**| *True/False* <br>Setting this to *True* enables Auto ML Classifier to perform preprocessing <br>on the input to handle *missing data*, and perform some common *feature extraction*|
|**max_cores_per_iteration**| Indicates how many cores on the compute target would be used to train a single pipeline.<br> Default is *1*, you can set it to *-1* to use all cores|

In [ ]:
automl_settings = {
    "max_time_sec": 12000,
    "iterations": 10,
    "n_cross_validations": 5,
    "primary_metric": 'AUC_weighted',
    "preprocess": True,
    "max_cores_per_iteration": 2
}

automl_config = AutoMLConfig(task = 'classification',
                             path=project_folder,
                             run_configuration = run_config,
                             data_script = project_folder + "./get_data.py",
                             **automl_settings
                            )


## Training the Model <a class="anchor" id="Training-the-model-Remote-DSVM"></a>

You can call the *fit* method on the AutoML instance and pass the dsvm runconfig name. For remote runs the execution is asynchronous, so you will see the iterations get populated as they complete. You can interact with the widgets/models even when the experiment is running to retreive the best model up to that point. Once you are satisfied with the model you can cancel a particular iteration or the whole run.


*fit* method on Auto ML Classifier triggers the training of the model. It can be called with the following parameters

**Note**: You cannot pass Numpy arrays directly to the fit method in case of remote executions.

|**Parameter**|**Description**|
|-|-|
|**compute_target**|Indicates the compute used for training. <i>local</i> indicates train on the same compute which hosts the jupyter notebook. <br>For DSVM and Batch AI please refer to the relevant notebooks.|
|**show_output**| True/False to turn on/off console output|

In [ ]:
remote_run = experiment.submit(automl_config)

## Exploring the Results <a class="anchor" id="Exploring-the-Results-Remote-DSVM"></a>
#### Widget for monitoring runs

The widget will sit on "loading" until the first iteration completed, then you will see an auto-updating graph and table show up. It refreshed once per minute, so you should see the graph update as child runs complete.

You can click on a pipeline to see run properties and output logs. Logs are also available on the DSVM under /tmp/azureml_run/{iterationid}/azureml-logs

NOTE: The widget displays a link at the bottom. This links to a web-ui to explore the individual run details.

In [ ]:
from azureml.train.widgets import RunDetails
RunDetails(remote_run).show() 


#### Retrieve All Child Runs
You can also use sdk methods to fetch all the child runs and see individual metrics that we log. 

In [ ]:
children = list(remote_run.get_children())
metricslist = {}
for run in children:
    properties = run.get_properties()
    metrics = {k: v for k, v in run.get_metrics().items() if isinstance(v, float)}    
    metricslist[int(properties['iteration'])] = metrics

rundata = pd.DataFrame(metricslist).sort_index(1)
cm = sns.light_palette("lightgreen", as_cmap = True)
s = rundata.style.background_gradient(cmap = cm)
s

## Canceling runs
You can cancel ongoing remote runs using the *cancel()* and *cancel_iteration()* functions

In [ ]:
# Cancel the ongoing experiment and stop scheduling new iterations
remote_run.cancel()

# Cancel iteration 1 and move onto iteration 2
# remote_run.cancel_iteration(1)

### Retrieve the Best Model

Below we select the best pipeline from our iterations. The *get_output* method on automl_classifier returns the best run and the fitted model for the last *fit* invocation. There are overloads on *get_output* that allow you to retrieve the best run and fitted model for *any* logged metric or a particular *iteration*.

In [ ]:
best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)

#### Best Model based on any other metric

In [ ]:
# lookup_metric = "accuracy"
# best_run, fitted_model = remote_run.get_output(metric=lookup_metric)

#### Best Model based on any iteration

In [ ]:
iteration = 0
best_run, fitted_model = remote_run.get_output(iteration=iteration)

### Register fitted model for deployment

In [ ]:
description = 'AutoML Model'
tags = None
remote_run.register_model(description=description, tags=tags)
remote_run.model_id # Use this id to deploy the model as a web service in Azure

### Testing the Fitted Model <a class="anchor" id="Testing-the-Fitted-Model-Remote-DSVM"></a>


In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from pandas_ml import ConfusionMatrix

df = pd.read_csv("https://automldemods.blob.core.windows.net/datasets/PlayaEvents2016,_1.6MB,_3.4k-rows.cleaned.2.tsv",
                     delimiter="\t", quotechar='"')

# get integer labels
le = LabelEncoder()
le.fit(df["Label"].values)
y = le.transform(df["Label"].values)
df = df.drop(["Label"], axis=1)

_, df_test, _, y_test = train_test_split(df, y, test_size=0.1, random_state=42)


ypred = fitted_model.predict(df_test.values)


ypred_strings = le.inverse_transform(ypred)
ytest_strings = le.inverse_transform(y_test)

cm = ConfusionMatrix(ytest_strings, ypred_strings)

print(cm)

cm.plot()